In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
import torchvision
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import numpy as np
from torch.optim.lr_scheduler import LambdaLR
#parameters

epochs=120
bs=128
lr=0.1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_save_name = 'scheduler.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"

#Create Model
def get_model():
 model=torchvision.models.resnet50(pretrained=False)
 return model,optim.SGD(model.parameters(),lr=lr,momentum=0.9,nesterov=True)
model,opt=get_model()
model.cuda()

#Checkpoint
checkpoint={}
#checkpoint=torch.load(path))
epoch_last=1

#model.load_state_dict(checkpoint['model_state_dict'])
#opt.load_state_dict(checkpoint['optimizer_state_dict'])
#epoch_last=checkpoint(['epoch'])

#lambda
def x(epoch):
  if epoch <=5:
     return epoch/5
  else:
     return 1
lambda1 = lambda epoch: x(epoch)
scheduler = LambdaLR(opt, [lambda1])

#accuracy
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return torch.sum(preds==yb).item()

#Loss function
loss_func=F.cross_entropy
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)
    print(loss)
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

   
#DATA LOADER  

train_dir=os.path.join('/content/gdrive/My Drive/imagenette2/','train')
val_dir=os.path.join('/content/gdrive/My Drive/imagenette2/','val')       
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
train_dataset = datasets.ImageFolder(train_dir,transforms.Compose([transforms.RandomResizedCrop(224),
transforms.RandomHorizontalFlip(),transforms.ToTensor(),normalize]))
train_loader=DataLoader(train_dataset,batch_size=bs,shuffle=True,pin_memory=True)
val_loader=DataLoader(datasets.ImageFolder(val_dir,transforms.Compose([transforms.Resize(256),
transforms.CenterCrop(224),transforms.ToTensor(),normalize])),batch_size=bs,shuffle=False,pin_memory=True)

for epoch in range(epoch_last,epochs):
        model.train()
        for xb, yb in train_loader:
            xb=xb.to('cuda')
            yb=yb.to('cuda')
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        losses=0
        nums=0
        correct=0
        with torch.no_grad():
            for xb, yb in val_loader:
              xb=xb.to('cuda')
              yb=yb.to('cuda')
              l,c=loss_batch(model, loss_func, xb, yb)
              losses +=l
              nums += c
              correct_class = accuracy(model(xb),yb)
              correct += correct_class
              print(correct)
        val_loss = losses / nums
        acc=correct/nums
        print(correct)
        print(epoch,acc)
        print(epoch, val_loss) 
        scheduler.step()
        print(scheduler.get_last_lr())
        torch.save({'epoch': epoch,'model_state_dict': model.state_dict(),'optimizer_state_dict': opt.state_dict(),'loss': val_loss,'ACCURACY':acc,'Correct':correct}, path)

Streaming output truncated to the last 5000 lines.
tensor(0.2361, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2436, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3113, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2511, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3284, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3456, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2363, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2426, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3184, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2929, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2735, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3757, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2606, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3213, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2640, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2315, device='cuda:0', grad_fn=<NllLossBackward>)
tenso

In [0]:
!kill -9 -1
!kill -9 -1
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 
